In [1]:
import pandas as pd
import nltk

In [2]:
# word tokenize will turn each word in a text string into a token
from nltk.tokenize import word_tokenize 

In [3]:
# punkt is a language aware model that can handle punctuation in text
nltk.download('punkt') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mccal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# i don't think there is any punctuation in the data that we are using to train this model but best to be safe

In [5]:
# there are definitely stop words in the data we will use to build the model (words that do not have sentiment)
from nltk.corpus import stopwords
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mccal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# this weighs the importance of a word based on freqeuency (feature extraction tool)
from sklearn.feature_extraction.text import TfidfVectorizer 

In [7]:
# we all know what this is
from sklearn.model_selection import train_test_split 

In [8]:
 # this is our model of choice
from sklearn.linear_model import LogisticRegression

In [9]:
# we will use this to see how accurate the model is
from sklearn.metrics import accuracy_score, classification_report 

In [10]:
# reading in labeled text data to make the model
emotions=pd.read_csv('emotions.csv') 

In [11]:
# basic data information
emotions.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    416809 non-null  object
 1   label   416809 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ MB


In [12]:
# here's what the data looks like
emotions.head() 

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [13]:
# sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5) emotions and their label

In [14]:
# we have some really unbalanced data here, this is what we adress first if we want to improve the model
emotions.label.value_counts() 

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [15]:
# i am writing a function that tokenizes text that we can apply to the text column
def tokenize(text):
    return nltk.word_tokenize(text.lower()) 

In [16]:
# applying the tokenizer function to create a tokens column
emotions['tokens']=emotions['text'].apply(tokenize) 

In [17]:
# this is going to be a set containing all of the english stop words
stop_words = set(stopwords.words('english')) 

In [18]:
# we are creating a column of tokens that do not contain any words we do not want to feed the model
emotions['filtered_tokens'] = emotions['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [19]:
 # here's what our data looks like now with the new features
emotions.head()

,text,label,tokens,filtered_tokens
0,i just feel really helpless and heavy hearted,4,"[i, just, feel, really, helpless, and, heavy, ...","[feel, really, helpless, heavy, hearted]"
1,ive enjoyed being able to slouch about relax a...,0,"[ive, enjoyed, being, able, to, slouch, about,...","[ive, enjoyed, able, slouch, relax, unwind, fr..."
2,i gave up my internship with the dmrg and am f...,4,"[i, gave, up, my, internship, with, the, dmrg,...","[gave, internship, dmrg, feeling, distraught]"
3,i dont know i feel so lost,0,"[i, dont, know, i, feel, so, lost]","[dont, know, feel, lost]"
4,i am a kindergarten teacher and i am thoroughl...,4,"[i, am, a, kindergarten, teacher, and, i, am, ...","[kindergarten, teacher, thoroughly, weary, job..."


In [20]:
# what's happening here is we are creating a column combining the filtered tokens back together into a text string
emotions['text_combined'] = emotions['filtered_tokens'].apply(lambda x: ' '.join(x))

In [21]:
# i mentioned earlier that this is going to make the text readable for the logistic regression  model
# this is going to create vectors from our text that we can use for training
# the vectors are made with term frequency, relative to amount of documents, which in turn tells us something about their importance
vectorizer = TfidfVectorizer()

In [22]:
# vectorize the text
X = vectorizer.fit_transform(emotions['text_combined']) 

In [23]:
y=emotions['label']

In [24]:
# splitting the data into training data and testing data
# 42 was chosen arbitrarily
# random state accepts any number in range [-2,147,483,648 , 2,147,483,647]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# Default is 'l2'
# i chose 200 for max_iter because it was large enough to reach convergence
# if you do not set the max_iter to 200, the default, 100 is not enough and you will get an error
model = LogisticRegression(penalty='l2', max_iter=200) 

In [26]:
# fit the model
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [27]:
# basic prediction function call
y_pred = model.predict(X_test)

In [28]:
accuracy = accuracy_score(y_test, y_pred)

In [29]:
# did pretty well
accuracy 

0.8939924665914926

In [30]:
# we aren't after the models prediction for the labels
# we are after the probability scores for each label because we want nuanced emotion vectors
# lets do a demo

In [31]:
# here is a string that i am going to write that will contain some things i want to say about my emotional state
hank_feeling = 'i had a stressfull day but i was able to get a lot done which made me proud'

In [32]:
# in order to pull an emotion vector from this string we need to apply the same principles we used to train the model
# we need to tokenize the text
# we need to filter the tokenized text for stopwords
# we need to re join that filtered token into plain text
# we need to extract features using tfidf vectorization
# we need to tell the model to predict probabilites NOT the label
# we're gonna investigate what the probabilities look like

In [33]:
#  recall this key; sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5) emotions and their label

In [34]:
# now let's write a function that does that whole process for us

In [35]:
def emotion_score(user_input):
    # tokenize the input
    input_tokens = nltk.word_tokenize(user_input.lower()) 
    # filter stopwords out of the input
    filtered_input_tokens = [word for word in input_tokens if word not in stop_words] 
    # re-join the filtered tokens
    input_combined = ' '.join(filtered_input_tokens) 
    # feature extraction
    input_vector = vectorizer.transform([input_combined]) 
    # pull the probabilities from the input
    probabilities = model.predict_proba(input_vector)
    # return the probabilities
    return probabilities.tolist()[0] 

In [36]:
emotion_score(hank_feeling) # test drive

[0.015336456086055822,
 0.9519273437595572,
 0.010170432898996905,
 0.009528920934266534,
 0.005507372710613149,
 0.007529473610510535]

In [37]:
# i wanna run this function on an entire data frame of song lyrics
music=pd.read_csv('songs_with_lyrics_Cleaned.csv') 

In [38]:
 # here's what the data frame with music data and song lyrics
music.head()

,Unnamed: 0,artist,song,link,text
0,0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"look at her face, it's a wonderful face and it..."
1,1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"take it easy with me, please touch me gently l..."
2,2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,i'll never know why i had to go why i had to p...
3,3,ABBA,Bang,/a/abba/bang_20598415.html,making somebody happy is a question of give an...
4,4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,making somebody happy is a question of give an...


In [39]:
# demo for how the song handles user input
emotion_score(' i had a pretty rough day, can you recommend me a sad song?')

[0.734118761988165,
 0.132520592501066,
 0.029530023819752265,
 0.040496651943053505,
 0.034728021670850914,
 0.028605948077112395]

In [40]:
# write a function that vectorizes the song lyric data with an emotion score dictionary in order to create a table for the lyric scores

In [41]:
def emotion_dictionary(user_input):
    # make tokens
    input_tokens = nltk.word_tokenize(user_input.lower()) 
    # filter the tokens
    filtered_input_tokens = [word for word in input_tokens if word not in stop_words] 
    # recombine filtered tokens
    input_combined = ' '.join(filtered_input_tokens) 
    # tfidf feature extraction
    input_vector = vectorizer.transform([input_combined]) 
    # call the probabilities
    probabilities = model.predict_proba(input_vector) 
    # create the vector
    emotion_vector = probabilities.tolist()[0] 
    # use the vector to create a dictionary with the scores and a key that references the scores
    return {'sadness': emotion_vector[0], 
    'joy': emotion_vector[1],
    'love': emotion_vector[2],
    'anger': emotion_vector[3],
    'fear': emotion_vector[4],
    'surprise': emotion_vector[5] }

In [42]:
# time to make the score table

In [43]:
# import tqdm for progress bar
from tqdm import tqdm 

In [44]:
# call this to get the progress bar
tqdm.pandas() 

In [45]:
# get a series containing the emotion scores for every song
scores=music['text'].progress_apply(emotion_dictionary).apply(pd.Series) 

100%|██████████| 44795/44795 [05:59<00:00, 124.65it/s]


In [46]:
# here is the scores data series that we will concat to our original data frame
scores 

,sadness,joy,love,anger,fear,surprise
0,0.037772,0.865948,0.020204,0.032962,0.023208,0.019906
1,0.066016,0.637793,0.148777,0.056284,0.058281,0.032849
2,0.557590,0.304692,0.064213,0.023887,0.027135,0.022482
3,0.136736,0.362437,0.262408,0.145773,0.065213,0.027435
4,0.967274,0.005258,0.016713,0.006844,0.002461,0.001449
...,...,...,...,...,...,...
44790,0.152381,0.504729,0.055719,0.141408,0.109148,0.036615
44791,0.426336,0.203087,0.030651,0.199676,0.091436,0.048814
44792,0.164597,0.422916,0.090274,0.126768,0.155709,0.039737
44793,0.297446,0.294825,0.074738,0.199254,0.100945,0.032792


In [47]:
# concat the scores to the music to get a df with the scores
scored_music=pd.concat([music,scores],axis=1) 

In [49]:
# useless column
scored_music.drop(columns=['Unnamed: 0'],inplace=True) 

In [50]:
# we also want a column with just the vectors
vector=music['text'].progress_apply(emotion_score) 

  0%|          | 0/44795 [00:00<?, ?it/s]

100%|██████████| 44795/44795 [06:29<00:00, 115.05it/s]


In [51]:
# create a column to store the vectors
scored_music['vector']=vector 

In [52]:
# our final dataframe
scored_music.sample(5) 

,artist,song,link,text,sadness,joy,love,anger,fear,surprise,vector
1290,Bette Midler,I Remember You,/b/bette+midler/i+remember+you_20017007.html,i remember you. you're the one who made my dre...,0.218151,0.231523,0.112770,0.110902,0.170540,0.156115,"[0.21815059097238804, 0.23152251377323413, 0.1..."
12655,Nirvana,Negative Creep,/n/nirvana/negative+creep_20100986.html,this is out of our reach this is out of our re...,0.302437,0.181382,0.052648,0.326273,0.115412,0.021848,"[0.30243722812567275, 0.18138196027271103, 0.0..."
40392,Scorpions,Destin,/s/scorpions/destin_20122670.html,do you say a prayer when your feeling down do ...,0.213674,0.572996,0.053409,0.073524,0.067277,0.019119,"[0.21367433070710073, 0.5729957097129348, 0.05..."
31336,Joni Mitchell,Ethiopia,/j/joni+mitchell/ethiopia_20075311.html,hot winds and hunger cries--ethiopia flies in ...,0.284354,0.239677,0.118459,0.245663,0.081239,0.030607,"[0.28435439380123806, 0.23967740983201632, 0.1..."
24808,Devo,Mr. B's Ballroom,/d/devo/mr+bs+ballroom_10084260.html,three cheers! they're yellin' again three chee...,0.307410,0.294094,0.106825,0.110291,0.131367,0.050013,"[0.30740997178756196, 0.2940941701538825, 0.10..."


In [53]:
# use pickle to pickle the variables instead of joblib

In [54]:
import pickle

In [55]:
#with open('emotion_model.pkl', 'wb') as file:
    #pickle.dump(model, file)

In [56]:
#with open('vectorizer.pkl','wb') as file:
    #pickle.dump(vectorizer,file)

In [57]:
#with open('scored_music.pkl','wb') as file:
    #pickle.dump(scored_music,file)